In [1]:
!pip install wandb -qqq
import wandb
wandb.login()

wandb: Currently logged in as: bhavya99 (use `wandb login --relogin` to force relogin)


True

In [3]:
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import log_loss
from keras.datasets import mnist

In [4]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
labels = ["T-shirt/Top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]
N , a , b = train_images.shape
#Dimesion of each Datapoint will be a*b
x = a*b
d = x
nl = len(labels)

#Question 1


In [5]:
def show_images(train_images , train_labels , desc):
  #run = wandb.init(entity="cs21m027_cs21m011", project="DL_ASG_1_final")
  #run.name = "Question_1"
  plot = []
  m=0
  n=0
  k=0
  fig, axes = plt.subplots(2, 5, figsize=(12.5,6))
  for i in range(1000):
    if train_labels[i]==k:
      ax = axes[m,n]
      ax.imshow(train_images[i], cmap='gray')
      ax.set_title('Class='+str(train_labels[i])+', Label='+str(desc[train_labels[i]]))
      plot.append(wandb.Image(train_images[i],caption= 'Class='+str(train_labels[i])+', Label='+str(desc[train_labels[i]])))
      n=n+1
      k=k+1
      if(n==5):
        m=1
        n=0
    if k==10:
      break
  plt.tight_layout()
  plt.show()
  wandb.log({"images":plot})

#Propecessing of data



In [6]:
#Flattening we are converting the 28*28 data point as 784*1 data point
flatted_train_images =  np.array([train_images[i].flatten() for i in range(len(train_images))])
#Same transition for testdata
flatted_test_images =  np.array([test_images[i].flatten() for i in range(len(test_images))])

In [7]:
#getting train data and validation data from training data
from sklearn.model_selection import train_test_split
train_x, valid_x, cat_train_y, cat_val_y = train_test_split(flatted_train_images, train_labels, test_size=0.1, stratify = train_labels ,random_state=42)

In [8]:
#One hot encoding of categorical labels 
def encode(data , nl):
  encode_data = np.array( [ [0]*nl for i in range(len(data))] )
  for i in range(len(data)):
    actual_label = data[i]
    encode_data[i][actual_label] = 1
  return encode_data

In [9]:
train_y = encode(cat_train_y , nl)
valid_y = encode(cat_val_y , nl)
test_y = encode(test_labels , nl)

In [10]:
#Normalizing the data
mean = np.mean(train_x , axis = 0)
train_x = (train_x - mean) /255
valid_x = (valid_x - mean)/255
test_x = (flatted_test_images - mean)/255

In [11]:
print(train_x.shape)
print(valid_x.shape)
print(test_x.shape)
print(train_y.shape)
print(valid_y.shape)
print(test_y.shape)

(54000, 784)
(6000, 784)
(10000, 784)
(54000, 10)
(6000, 10)
(10000, 10)


#Activation functions


In [12]:
def sigmoid(x):
    temp = []
    for i in range(len(x)):
      temp.append(1/(1 + np.exp(-(float(x[i])))))
    return np.array(temp)

def tanh(x):
    temp = []
    for i in range(len(x)):
      temp.append(np.tanh(x[i]))
    return np.array(temp)

def relu(x):
    temp = []
    for i in range(len(x)):
      temp.append(max(0,x[i]))
    return np.array(temp)

def softmax(x):
    temp = []
    sum = 0
    for i in range(len(x)):
      sum += np.exp(float(x[i]))
    for i in range(len(x)):
      temp.append( np.exp(float(x[i])) / sum)
    return np.array(temp)

def derivative_sigmoid(x):
    return x*(1 - x)

def derivative_tanh(x):
    return 1-np.square(x)

def derivative_relu(x):
    temp = []
    for i in range(len(x)):
      if x[i]>0:
        temp.append(1)
      else:
        temp.append(0)
    return np.array(temp)

#Network initialization

In [13]:
def initialize_zeros(dim,hl,ol):
  w = [np.array([])]
  b = [np.array([])]
  for i in range(len(hl)):
    b.append(np.zeros(hl[i]))
    if(i == 0):
      w.append(np.zeros((hl[i],d)))
    else:
      w.append(np.zeros((hl[i],hl[i - 1])))

  for i in range(len(ol)):
    b.append(np.zeros(ol[i]))
    w.append(np.zeros((ol[i],hl[-1])))
  return w , b

In [14]:
def initialize_network(dim, hl, ol, method):
  W = [np.array([])]
  b = [np.array([])]
  np.random.seed(2)
  #Random intialization
  if(method=='random'):
    for i in range(len(hl)):
      b.append(np.random.randn(hl[i]))
      if(i == 0):
        W.append(np.random.randn(hl[i],dim))
      else:
        W.append(np.random.randn(hl[i],hl[i - 1]))
    for i in range(len(ol)):
      b.append(np.random.randn(ol[i]))
      W.append(np.random.randn(ol[i],hl[-1]))
      
  #Xavier Initialization
  else:
    for i in range(len(hl)):
      b.append(np.random.randn(hl[i]))
      if(i == 0):
        W.append(np.random.randn(hl[i],dim ) * np.sqrt(1/dim))
      else:
        W.append(np.random.randn(hl[i],hl[i-1]) * np.sqrt(1/hl[i-1]))
    for i in range(len(ol)):
      b.append(np.random.randn(ol[i]))
      W.append(np.random.randn(ol[i],hl[-1]) * np.sqrt(1/hl[-1]))

  return W,b

#Forward Propagation

In [15]:
def forward_propagation(W,b,x,method):
    a = [[]]
    h = [[]]
    h[0] = x 
    num_layers = (len(W)-1)
    c = []
    d = []
    if method=='sigmoid':
        for i in range(1 , num_layers):
            c = np.dot( W[i], h[i-1] ) + b[i]
            a.append(c)
            d = sigmoid(c)
            h.append(d)

    elif method=='tanh':
        for i in range(1 , num_layers):
            c = np.dot( W[i], h[i-1] ) + b[i]
            a.append(c)
            d = tanh(c)
            h.append(d)

    elif method=='relu':
        for i in range(1 , num_layers):
            c = np.dot( W[i], h[i-1] ) + b[i]
            a.append(c)
            d = relu(c)
            h.append(d)

    c = np.dot( W[num_layers], h[num_layers-1] ) + b[num_layers]
    a.append(c)
    d = softmax(c)
    h.append(d)

    return a,h

#Backward Propagation

In [16]:
def back_prop(W,h,x,y,y_pred,act_fun,loss_fun):
  del_W,del_b=[[]],[[]]

  #Computing output grad wrt Cross Entropy Loss function
  if loss_fun == "cross_entropy" :
    del_a = (y_pred - y)
    
  #Computing output grad wrt Squared Error Loss function
  else:
    y_label = y_pred[np.argmax(y)]
    del_a = 2 * (y_label - 1) * y_label * ( y - y_pred )

  for i in range(len(W)-1, 0, -1):

    #computing gradients wrt parameters W,b
    dW = np.array(np.dot(np.matrix(del_a).T , np.matrix(h[i-1])))
    db = np.array( del_a )

    #computing gradients wrt below layer activation function
    dh = np.dot( np.transpose(W[i]), del_a )

    #computing gradients wrt below layer pre-activation function
    if act_fun == "sigmoid":
      del_a = dh * derivative_sigmoid( h[i - 1] )
    
    elif act_fun == "tanh":
      del_a = dh * derivative_tanh( h[i - 1] )
    
    elif act_fun == "relu":
      del_a = dh * derivative_relu( h[i - 1] )

    del_W.insert(1, dW)
    del_b.insert(1, db)

  return del_W, del_b

#Stochastic Gradient Descent optimizer

In [17]:
def stochastic_gradient_descent(train_x,train_y,valid_x,valid_y,d,hl,ol,act_fun,loss_fun,epochs,eta,strat,alpha,batch):
  #Initializing the weights and biases based on given strategy
  W,b = initialize_network(d, hl, ol, strat)
  dw , db = initialize_zeros(784,hl,ol)

  seen = 0
  for e in range(epochs):
    
    for it, (x, y) in enumerate(zip(train_x, train_y)):
      seen += 1
      a,h = forward_propagation(W, b, x, act_fun)
      y_pred=h[len(h)-1]
      tdw,tdb=back_prop(W, h, x, y, y_pred, act_fun, loss_fun)

      for i in range(len(tdw)):
        dw[i] += tdw[i]
        db[i] += tdb[i]

      if(seen==batch or it==len(train_x)-1):
        seen = 0

        for i, (weight, deriv) in enumerate(zip(W, dw)):
          W[i] = weight - eta * np.array(deriv)

        for i, (bias, deriv) in enumerate(zip(b, db)):
          b[i] = bias - eta * np.array(deriv)
        
        dw , db = initialize_zeros(784,hl,ol)

    val_acc, val_loss,yt_val,ypred_train = get_predictions_accuracy(W, b, valid_x, valid_y, act_fun, loss_fun)
    train_acc, train_loss,yt_train,ypred_train = get_predictions_accuracy(W, b, train_x, train_y, act_fun, loss_fun)
    test_acc, test_loss,yt_test,ypred_test = get_predictions_accuracy(W, b, test_x, test_y, act_fun, loss_fun)

    print("epoch:" , e+1 , " "+loss_fun+" ", "train_acc :" , train_acc , "valid_acc :" , val_acc , "test_acc :" , test_acc)
    
    if loss_fun=="cross_entropy":
      wandb.log({
          "Epoch": e+1,
          "Train Loss": train_loss,
          "Train Acc": train_acc,
          "Valid Loss": val_loss,
          "Valid Acc": val_acc})
    else:
      wandb.log({
          "Train Loss (squared_error)": train_loss,
          "Train Acc (squared_error)": train_acc,
          "Valid Loss (squared_error)": val_loss,
          "Valid Acc (squared_error)": val_acc})
  return W,b

#Momentum Based Gradient Descent 

In [18]:
def momentum_gradient_descent(train_x,train_y,valid_x,valid_y,d,hl,ol,act_fun,loss_fun,epochs,eta,strat,alpha,batch):
  W,b=initialize_network(d, hl, ol, strat)
  dw , db = initialize_zeros(784,hl,ol)
  prev_w , prev_b = initialize_zeros(784,hl,ol)
  gamma=0.9
  seen=0

  for e in range(epochs):
    #dw , db = initialize_zeros(784,hl,ol)
    for it, (x, y) in enumerate(zip(train_x, train_y)):
      seen+=1
      a,h = forward_propagation(W, b, x, act_fun)
      y_pred=h[len(h)-1]
      
      tdw,tdb=back_prop(W, h, x, y, y_pred, act_fun, loss_fun)
  
      for i in range(len(tdw)):
        dw[i] += tdw[i]
        db[i] += tdb[i]
    
      if(seen==batch or it==len(train_x)-1):
        seen=0
        for i, (weight, deriv) in enumerate(zip(W, dw)):
          W[i] = weight - eta * np.array(deriv) - gamma*prev_w[i]
          prev_w[i] = eta * np.array(deriv) + gamma*prev_w[i]

        for i, (bias, deriv) in enumerate(zip(b, db)):
          b[i] = bias - eta * np.array(deriv) - gamma*prev_b[i]
          prev_b[i] = eta * np.array(deriv) + gamma*prev_b[i]

        dw , db = initialize_zeros(784,hl,ol)

    val_acc, val_loss,yt_val,ypred_train = get_predictions_accuracy(W, b, valid_x, valid_y, act_fun, loss_fun)
    train_acc, train_loss,yt_train,ypred_train = get_predictions_accuracy(W, b, train_x, train_y, act_fun, loss_fun)
    test_acc, test_loss,yt_test,ypred_test = get_predictions_accuracy(W, b, test_x, test_y, act_fun, loss_fun)

    print("epoch:" , e+1 , " "+loss_fun+" ", "train_acc :" , train_acc , "valid_acc :" , val_acc , "test_acc :" , test_acc)
    
    if loss_fun=="cross_entropy":
      wandb.log({
          "Epoch": e+1,
          "Train Loss": train_loss,
          "Train Acc": train_acc,
          "Valid Loss": val_loss,
          "Valid Acc": val_acc})
    else:
      wandb.log({
          "Train Loss (squared_error)": train_loss,
          "Train Acc (squared_error)": train_acc,
          "Valid Loss (squared_error)": val_loss,
          "Valid Acc (squared_error)": val_acc})
  return W,b

#Nesterov Accelerated Gradient Descent

In [19]:
def nesterov_gradient_descent(train_x,train_y,valid_x,valid_y,d,hl,ol,act_fun,loss_fun,epochs,eta,strat,alpha,batch):
  W,b=initialize_network(d, hl, ol, strat)

  gamma=0.9
  seen=0

  
  prev_w , prev_b = initialize_zeros(784,hl,ol)
  v_w , v_b =initialize_zeros(784,hl,ol)

  dw , db = initialize_zeros(784,hl,ol)
  tw , tb = initialize_zeros(784,hl,ol)

  for e in range(epochs):

    for i in range(len(W)):
      v_w[i] = gamma*prev_w[i]
      v_b[i] = gamma*prev_b[i]
    

    for i in range(len(W)):
      tw[i] = W[i] - v_w[i]
      tb[i] = b[i] - v_b[i]
  
    for it, (x, y) in enumerate(zip(train_x, train_y)):
      seen+=1

      a,h = forward_propagation(tw, tb, x, act_fun)
      y_pred=h[len(h)-1]
      
      tdw,tdb=back_prop(tw, h, x, y, y_pred, act_fun, loss_fun)
      
      for i in range(len(tdw)):
        dw[i] += tdw[i]
        db[i] += tdb[i]

      if seen==batch or it == len(train_x)-1:
        seen=0
        for i, (weight, deriv) in enumerate(zip(W, dw)):
          v_w[i] = gamma*prev_w[i] + eta*np.array(deriv)
          W[i] = weight - v_w[i]
          tw[i] = W[i]
          prev_w = v_w

        for i, (bias, deriv) in enumerate(zip(b, db)):
          v_b[i] = gamma*prev_b[i] + eta*np.array(deriv)
          b[i] = bias - v_b[i]
          tb[i] = b[i]
          prev_b = v_b
        
        dw , db = initialize_zeros(784,hl,ol)

    val_acc, val_loss,yt_val,ypred_train = get_predictions_accuracy(W, b, valid_x, valid_y, act_fun, loss_fun)
    train_acc, train_loss,yt_train,ypred_train = get_predictions_accuracy(W, b, train_x, train_y, act_fun, loss_fun)
    test_acc, test_loss,yt_test,ypred_test = get_predictions_accuracy(W, b, test_x, test_y, act_fun, loss_fun)

    print("epoch:" , e+1 , " "+loss_fun+" ", "train_acc :" , train_acc , "valid_acc :" , val_acc , "test_acc :" , test_acc)
    
    if loss_fun=="cross_entropy":
      wandb.log({
          "Epoch": e+1,
          "Train Loss": train_loss,
          "Train Acc": train_acc,
          "Valid Loss": val_loss,
          "Valid Acc": val_acc})
    else:
      wandb.log({
          "Train Loss (squared_error)": train_loss,
          "Train Acc (squared_error)": train_acc,
          "Valid Loss (squared_error)": val_loss,
          "Valid Acc (squared_error)": val_acc})
  return W,b

#RMS_Prop Optimizer

In [23]:
def rmsprop(train_x,train_y,valid_x,valid_y,d,hl,ol,act_fun,loss_fun,epochs,eta,strat,alpha,batch):
  W,b=initialize_network(d, hl, ol, strat)
  seen=0
  
  prev_w , prev_b = initialize_zeros(784,hl,ol)
  dw , db = initialize_zeros(784,hl,ol)
  eps , beta = 1e-8 , 0.9

  for e in range(epochs):
  
    for it, (x, y) in enumerate(zip(train_x, train_y)):
      seen+=1
      a,h = forward_propagation(W, b, x, act_fun)
      y_pred=h[len(h)-1]
      
      tdw,tdb=back_prop(W, h, x, y, y_pred, act_fun, loss_fun)
      
      for i in range(len(tdw)):
        dw[i] += tdw[i]
        db[i] += tdb[i]

      if seen==batch or it == len(train_x)-1:

        seen=0

        for i in range(len(W)):
          prev_w[i] = beta*prev_w[i] + (1-beta)*(dw[i]**2)
          prev_b[i] = beta*prev_b[i] + (1-beta)*(db[i]**2)

        for i, (weight, deriv) in enumerate(zip(W, dw)):
          W[i] = weight - (eta / np.sqrt(prev_w[i] + eps)) * np.array(deriv)

        for i, (bias, deriv) in enumerate(zip(b, db)):
          b[i] = bias - (eta / np.sqrt(prev_b[i] + eps)) * np.array(deriv)
        
        dw , db = initialize_zeros(784,hl,ol)

    val_acc, val_loss,yt_val,ypred_train = get_predictions_accuracy(W, b, valid_x, valid_y, act_fun, loss_fun)
    train_acc, train_loss,yt_train,ypred_train = get_predictions_accuracy(W, b, train_x, train_y, act_fun, loss_fun)
    test_acc, test_loss,yt_test,ypred_test = get_predictions_accuracy(W, b, test_x, test_y, act_fun, loss_fun)

    print("epoch:" , e+1 , " "+loss_fun+" ", "train_acc :" , train_acc , "valid_acc :" , val_acc , "test_acc :" , test_acc)
    
    '''if loss_fun=="cross_entropy":
      wandb.log({
          "Epoch": e+1,
          "Train Loss": train_loss,
          "Train Acc": train_acc,
          "Valid Loss": val_loss,
          "Valid Acc": val_acc})
    else:
      wandb.log({
          "Train Loss (squared_error)": train_loss,
          "Train Acc (squared_error)": train_acc,
          "Valid Loss (squared_error)": val_loss,
          "Valid Acc (squared_error)": val_acc})'''
  return W,b

#Adam Optimizer

In [32]:
def adaptive_moments(train_x,train_y,valid_x,valid_y,d,hl,ol,act_fun,loss_fun,epochs,eta,strat,alpha,batch):
  W,b=initialize_network(d, hl, ol, strat)
  seen=0
  c=0
  v_w , v_b = initialize_zeros(784,hl,ol)
  dw , db = initialize_zeros(784,hl,ol)
  m_w , m_b = initialize_zeros(784,hl,ol)
  eps , beta1 , beta2 = 1e-8 , 0.9 , 0.99

  m_w_hat , m_b_hat = initialize_zeros(784,hl,ol)
  v_w_hat , v_b_hat = initialize_zeros(784,hl,ol)

  for e in range(epochs):
  
    for it, (x, y) in enumerate(zip(train_x, train_y)):
      seen+=1
      a,h = forward_propagation(W, b, x, act_fun)
      y_pred=h[len(h)-1]
      
      tdw,tdb=back_prop(W, h, x, y, y_pred, act_fun, loss_fun)
      
      for i in range(len(tdw)):
        dw[i] += tdw[i]
        db[i] += tdb[i]

      if seen==batch or it == len(train_x)-1:

        seen=0
        c+=1
        for i in range(len(W)):
          m_w[i] = beta1 * m_w[i] + (1-beta1) * dw[i]
          m_b[i] = beta1 * m_b[i] + (1-beta1) * db[i]

          v_w[i] = beta2 * v_w[i] + (1-beta2) * (dw[i]**2)
          v_b[i] = beta2 * v_b[i] + (1-beta2) * (db[i]**2)

        for i in range(len(W)):
          m_w_hat[i] = m_w[i] / (1 - np.power(beta1,c))
          m_b_hat[i] = m_b[i] / (1 - np.power(beta1,c))

          v_w_hat[i] = v_w[i] / (1 - np.power(beta2,c))
          v_b_hat[i] = v_b[i] / (1 - np.power(beta2,c))

        for i, (weight, deriv) in enumerate(zip(W, dw)):
          W[i] = weight - (eta / np.sqrt(v_w_hat[i] + eps)) * m_w_hat[i]

        for i, (bias, deriv) in enumerate(zip(b, db)):
          b[i] = bias - (eta / np.sqrt(v_b_hat[i] + eps)) * m_b_hat[i]
        
        dw , db = initialize_zeros(784,hl,ol)

    val_acc, val_loss,yt_val,ypred_train = get_predictions_accuracy(W, b, valid_x, valid_y, act_fun, loss_fun)
    train_acc, train_loss,yt_train,ypred_train = get_predictions_accuracy(W, b, train_x, train_y, act_fun, loss_fun)
    test_acc, test_loss,yt_test,ypred_test = get_predictions_accuracy(W, b, test_x, test_y, act_fun, loss_fun)

    print("epoch:" , e+1 , " "+loss_fun+" ", "train_acc :" , train_acc , "valid_acc :" , val_acc , "test_acc :" , test_acc)
    
    '''if loss_fun=="cross_entropy":
      wandb.log({
          "Epoch": e+1,
          "Train Loss": train_loss,
          "Train Acc": train_acc,
          "Valid Loss": val_loss,
          "Valid Acc": val_acc})
    else:
      wandb.log({
          "Train Loss (squared_error)": train_loss,
          "Train Acc (squared_error)": train_acc,
          "Valid Loss (squared_error)": val_loss,
          "Valid Acc (squared_error)": val_acc})'''
  return W,b

#Nadam Optimizer

In [25]:
def nadam(train_x,train_y,valid_x,valid_y,d,hl,ol,act_fun,loss_fun,epochs,eta,strat,alpha,batch):
  W,b=initialize_network(d, hl, ol, strat)
  seen=0
  c=0
  v_w , v_b = initialize_zeros(784,hl,ol)
  dw , db = initialize_zeros(784,hl,ol)
  m_w , m_b = initialize_zeros(784,hl,ol)
  eps , beta1 , beta2 = 1e-8 , 0.9 , 0.99

  m_w_hat , m_b_hat = initialize_zeros(784,hl,ol)
  v_w_hat , v_b_hat = initialize_zeros(784,hl,ol)

  for e in range(epochs):
  
    for it, (x, y) in enumerate(zip(train_x, train_y)):
      seen+=1
      a,h = forward_propagation(W, b, x, act_fun)
      y_pred=h[len(h)-1]
      
      tdw,tdb=back_prop(W, h, x, y, y_pred, act_fun, loss_fun)
      
      for i in range(len(tdw)):
        dw[i] += tdw[i]
        db[i] += tdb[i]

      if seen==batch or it == len(train_x)-1:

        seen=0
        c+=1
        for i in range(len(W)):
          m_w[i] = beta1 * m_w[i] + (1-beta1) * dw[i]
          m_b[i] = beta1 * m_b[i] + (1-beta1) * db[i]

          v_w[i] = beta2 * v_w[i] + (1-beta2) * (dw[i]**2)
          v_b[i] = beta2 * v_b[i] + (1-beta2) * (db[i]**2)

        for i in range(len(W)):
          m_w_hat[i] = ( (beta1 * m_w[i]) + ( (1 - beta1) * dw[i] ) )/ (1 - np.power(beta1,c))
          m_b_hat[i] = ( (beta1 * m_b[i]) + ( (1 - beta1) * db[i] ) )/ (1 - np.power(beta1,c))

          v_w_hat[i] = v_w[i] / (1 - np.power(beta2,c))
          v_b_hat[i] = v_b[i] / (1 - np.power(beta2,c))

        for i, (weight, deriv) in enumerate(zip(W, dw)):
          W[i] = weight - (eta / np.sqrt(v_w_hat[i] + eps)) * m_w_hat[i]

        for i, (bias, deriv) in enumerate(zip(b, db)):
          b[i] = bias - (eta / np.sqrt(v_b_hat[i] + eps)) * m_b_hat[i]
        
        dw , db = initialize_zeros(784,hl,ol)

    val_acc, val_loss,yt_val,ypred_train = get_predictions_accuracy(W, b, valid_x, valid_y, act_fun, loss_fun)
    train_acc, train_loss,yt_train,ypred_train = get_predictions_accuracy(W, b, train_x, train_y, act_fun, loss_fun)
    test_acc, test_loss,yt_test,ypred_test = get_predictions_accuracy(W, b, test_x, test_y, act_fun, loss_fun)

    print("epoch:" , e+1 , " "+loss_fun+" ", "train_acc :" , train_acc , "valid_acc :" , val_acc , "test_acc :" , test_acc)
    
    '''if loss_fun=="cross_entropy":
      wandb.log({
          "Epoch": e+1,
          "Train Loss": train_loss,
          "Train Acc": train_acc,
          "Valid Loss": val_loss,
          "Valid Acc": val_acc})
    else:
      wandb.log({
          "Train Loss (squared_error)": train_loss,
          "Train Acc (squared_error)": train_acc,
          "Valid Loss (squared_error)": val_loss,
          "Valid Acc (squared_error)": val_acc})'''
  return W,b

#Confusion_matrix

In [26]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.express as px
import pandas as pd
def get_confusion_matrix(y_true , y_pred , class_names , figsize=(20,20)):
    cm = confusion_matrix(y_true, y_pred, labels=class_names)
    fontsize = 14
    r=class_names[::-1]
    #z_text = [[str(y_true) for y_true in y_pred] for y_pred in cm]

    df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)

    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    h = [ [0 for _ in range(len(class_names))] for _ in range(len(class_names)) ]
    #h = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%% <br> %d/%d' % (p, c, s)
                h[i][j] = '%d %s are correctly classified' % (c,labels[i])
            elif c == 0:
                annot[i, j] = ''
                h[i][j] = ''
            else:
                annot[i, j] = '%.1f%% <br> %d' % (p, c)
                h[i][j] = '%d %s are wrongly classified as %s' % (c,labels[i],labels[j])

    fig = ff.create_annotated_heatmap(cm , x = class_names, y = class_names , text=h,annotation_text = annot , hoverinfo='text' ,colorscale ='blugrn_R')
    fig['layout']['yaxis']['autorange'] = "reversed"
    
   
    # add custom xaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),x=0.5,y=-0.15,showarrow=False,text="Predicted value",xref="paper",yref="paper"))

    # add custom yaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),x=-0.35,y=0.5,showarrow=False,text="Real value",textangle=-90,xref="paper",yref="paper"))

    # adjust margins to make room for yaxis title
    fig.update_layout(margin=dict(t=50, l=200))
    fig.update_annotations(font_size=14)
   
    fig['data'][0]['showscale'] = True
    fig.update_xaxes(side="top")
    fig.show()
    return fig

#Predictions and accuracy function

In [28]:
def get_predictions_accuracy(W, b, X, y, method, loss_fun):
  sum,loss=0,0
  yhat = []
  yt = []
  for dp in range(len(X)):
    a = []
    h = []
    h = X[dp] 
    num_layers = (len(W)-1)
    if method=='sigmoid':
      for i in range(1 , num_layers):
        a = np.dot( W[i], h ) + b[i]
        h = sigmoid(a)

    elif method=='tanh':
      for i in range(1 , num_layers):
        a = np.dot( W[i], h ) + b[i]
        h = tanh(a)

    elif method=='relu':
      for i in range(1 , num_layers):
        a = np.dot( W[i], h ) + b[i]
        h = relu(a)

    a = np.dot( W[num_layers], h ) + b[num_layers]
    y_pred = softmax(a)

    ytrue = y[dp]
    if(ytrue[np.argmax(y_pred)]==1):
      sum=sum+1
    
    if loss_fun == "cross_entropy":
      loss += -np.sum(ytrue*np.log(y_pred))
    else:
      loss += np.sum((ytrue-y_pred)**2)

    yhat.append(labels[np.argmax(y_pred)])
    yt.append(labels[np.argmax(ytrue)])

  acc=sum/len(X)
  loss=loss/len(X)

  return acc,loss,yt,yhat


# Best Model 1 on MNIST

In [30]:
hidden_layer_size = 128
hidden_layers = 2
hl = [hidden_layer_size]*hidden_layers
ol = [len(train_y[0])]
n_hl = len(hl)
act_fun ='relu'
loss_fun='cross-entropy'
eta=0.001
alpha=0
strat='xavier'
epochs=5
batch = 64
W,b = nadam(train_x,train_y,valid_x,valid_y,d,hl,ol,act_fun,loss_fun,epochs,eta,strat,alpha,batch)

epoch: 1  cross-entropy  train_acc : 0.9593888888888888 valid_acc : 0.9496666666666667 test_acc : 0.9547
epoch: 2  cross-entropy  train_acc : 0.9711111111111111 valid_acc : 0.9611666666666666 test_acc : 0.9634
epoch: 3  cross-entropy  train_acc : 0.9761851851851852 valid_acc : 0.9655 test_acc : 0.9645
epoch: 4  cross-entropy  train_acc : 0.9799444444444444 valid_acc : 0.9665 test_acc : 0.9682
epoch: 5  cross-entropy  train_acc : 0.9839074074074075 valid_acc : 0.9708333333333333 test_acc : 0.9696


# Best Model 2 MNIST 

In [35]:
hidden_layer_size = 64
hidden_layers = 2
hl = [hidden_layer_size]*hidden_layers
ol = [len(train_y[0])]
n_hl = len(hl)
act_fun ='relu'
loss_fun='cross-entropy'
eta=0.001
alpha=0
strat='xavier'
epochs=5
batch = 64
W,b = adaptive_moments(train_x,train_y,valid_x,valid_y,d,hl,ol,act_fun,loss_fun,epochs,eta,strat,alpha,batch)

epoch: 1  cross-entropy  train_acc : 0.9534074074074074 valid_acc : 0.9426666666666667 test_acc : 0.9485
epoch: 2  cross-entropy  train_acc : 0.9653333333333334 valid_acc : 0.9528333333333333 test_acc : 0.9564
epoch: 3  cross-entropy  train_acc : 0.9743703703703703 valid_acc : 0.9633333333333334 test_acc : 0.9628
epoch: 4  cross-entropy  train_acc : 0.9785 valid_acc : 0.9645 test_acc : 0.9653
epoch: 5  cross-entropy  train_acc : 0.9808518518518519 valid_acc : 0.9678333333333333 test_acc : 0.9679


# Best Model 3 MNIST

In [36]:
hidden_layer_size = 128
hidden_layers = 5
hl = [hidden_layer_size]*hidden_layers
ol = [len(train_y[0])]
n_hl = len(hl)
act_fun ='relu'
loss_fun='cross-entropy'
eta=0.001
alpha=0
strat='xavier'
epochs=5
batch = 32
W,b = rmsprop(train_x,train_y,valid_x,valid_y,d,hl,ol,act_fun,loss_fun,epochs,eta,strat,alpha,batch)

epoch: 1  cross-entropy  train_acc : 0.9325925925925926 valid_acc : 0.9253333333333333 test_acc : 0.928
epoch: 2  cross-entropy  train_acc : 0.9621666666666666 valid_acc : 0.956 test_acc : 0.9566
epoch: 3  cross-entropy  train_acc : 0.9650925925925926 valid_acc : 0.9558333333333333 test_acc : 0.9583
epoch: 4  cross-entropy  train_acc : 0.9570925925925926 valid_acc : 0.9481666666666667 test_acc : 0.9501
epoch: 5  cross-entropy  train_acc : 0.9736296296296296 valid_acc : 0.9626666666666667 test_acc : 0.9639
